# Deep Learning Home Work 2

### Installing Required libraries

In [1]:
!pip install transformers datasets evaluate huggingface_hub rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 427.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 379.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.0 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=09c1d7bccb8d80865557595a62a93b34a78aa95b7d78818c4190c711d769bd9f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's d

## Import Required Libraries

In [1]:
from huggingface_hub import login
from google.colab import userdata
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
import evaluate
import numpy as np
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, BartForConditionalGeneration
import torch
from tqdm import tqdm

### Hugging Face Login


*   Setup your huggingface token as user secret in colab environment
*   Perform huggingface login, if you want to save your model on huggingface.
i.e. uncomment below cell to save model in your huggingface account.



In [2]:
login(token=userdata.get('hf_token'), add_to_git_credential=True)

## Load Dataset

In [3]:
samsum = load_dataset("knkarthick/samsum", split="train[:4000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
test_dataset = load_dataset("knkarthick/samsum", split="test[:500]")

In [5]:
samsum

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 4000
})

In [6]:
test_dataset

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 500
})

### Split the validation dataset into train and test.

In [7]:
samsum = samsum.train_test_split(test_size=0.2)

In [8]:
samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 3200
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 800
    })
})

In [9]:
samsum["train"][0]

{'id': '13821614',
 'dialogue': "Janek: Przemek is pissing me off\nJanek: Look\nJanek: <file_photo> \nMilena: Who is that fucking asshole? \nJanek: He worked at our company\nJanek: In the IT section\nMilena: I dont know him \nMilena: But sounds like a douche\nJanek: He doesn't leave me alone\nJanek: Hes a hypocrite and idk what to write him back\nJanek: He pisses me off all the time and then he tries to say that he was just kidding \nMilena: Ignore that piece of shit ",
 'summary': 'Przemek from the IT department annoys Janek. Milena tells Janek to ignore Przemek.'}

## Helper Functions


*   preprocess_function : Creates tokenized model inputs and labels.
*   compute_metrics: Function to calculate evaluation metrics.
*   evaluate_model_batch: Function to perform model evaluation on batch data.



In [10]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["dialogue"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [11]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
import torch
def evaluate_model_batch(model, dataset, tokenizer, data_collator, batch_size=16, prefix="summarize: "):
    model.eval()
    rouge = evaluate.load("rouge")
    all_predictions = []
    all_references = []
    total_gen_len = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # Create a PyTorch DataLoader
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=lambda batch: {
            'dialogue': [item['dialogue'] for item in batch],
            'summary': [item['summary'] for item in batch]
        }
    )

    device = model.device

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = [prefix + doc for doc in batch['dialogue']]
            model_inputs = tokenizer(inputs, max_length=1024, padding=True, truncation=True, return_tensors="pt")
            model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

            outputs = model.generate(**model_inputs, max_length=128, num_return_sequences=1)

            decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            all_predictions.extend(decoded_outputs)
            all_references.extend(batch['summary'])

            total_gen_len += sum(len(output.split()) for output in decoded_outputs)

    # Compute ROUGE scores
    rouge_scores = rouge.compute(predictions=all_predictions, references=all_references, use_stemmer=True)

    # Add average generation length
    rouge_scores['gen_len'] = total_gen_len / len(all_predictions)

    return {k: round(v, 4) for k, v in rouge_scores.items()}

In [13]:
import torch
from tqdm import tqdm

def evaluate_model_batch(model, dataset, tokenizer, data_collator, batch_size=16, prefix="summarize: "):
    model.eval()
    rouge = evaluate.load("rouge")
    all_predictions = []
    all_references = []
    total_gen_len = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to device
    model = model.to(device)

    # Create a PyTorch DataLoader
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=lambda batch: {
            'dialogue': [item['dialogue'] for item in batch],
            'summary': [item['summary'] for item in batch]
        }
    )

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            inputs = [prefix + doc for doc in batch['dialogue']]
            model_inputs = tokenizer(inputs, max_length=1024, padding=True, truncation=True, return_tensors="pt")

            # Move input data to device
            model_inputs = {k: v.to(device) for k, v in model_inputs.items()}

            outputs = model.generate(**model_inputs, max_length=128, num_return_sequences=1)

            decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            all_predictions.extend(decoded_outputs)
            all_references.extend(batch['summary'])

            total_gen_len += sum(len(output.split()) for output in decoded_outputs)

    # Compute ROUGE scores
    rouge_scores = rouge.compute(predictions=all_predictions, references=all_references, use_stemmer=True)

    # Add average generation length
    rouge_scores['gen_len'] = total_gen_len / len(all_predictions)

    return {k: round(v, 4) for k, v in rouge_scores.items()}

In [14]:
rouge = evaluate.load("rouge")

## Model 1 - (t5-small)

### Load tokenizer

In [17]:
checkpoint = "t5-small" #https://huggingface.co/t5-small
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

### Perform data preprocessing, convert text data to tokenized version

In [18]:
tokenized_samsum = samsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

### Create data collator

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### Load pretrained model from checkpoint

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Evaluating model without training on test data

In [21]:
# Evaluate the model without training on test data
print("Evaluating model without training on test data:")
results_without_training = evaluate_model_batch(model, test_dataset, tokenizer, data_collator)
print(results_without_training)

Evaluating model without training on test data:


Evaluating: 100%|██████████| 32/32 [10:53<00:00, 20.42s/it]


{'rouge1': 0.2392, 'rouge2': 0.0609, 'rougeL': 0.1894, 'rougeLsum': 0.1898, 'gen_len': 24.008}


### Perform Training and evaluate model after training

In [24]:
training_args = Seq2SeqTrainingArguments(
    output_dir="samsum_model_t5_small_10_epochs",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_samsum["train"],
    eval_dataset=tokenized_samsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-24-44abcc3f5a38>:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.952826,0.384400,0.156700,0.318400,0.318200,16.136200
2,No log,1.922138,0.388500,0.161300,0.321200,0.321000,16.332500
3,2.099600,1.907222,0.393600,0.166100,0.326400,0.325900,16.228800
4,2.099600,1.892993,0.398400,0.167800,0.329500,0.329200,16.337500
5,2.029700,1.885963,0.400500,0.170800,0.333000,0.332900,16.355000
6,2.029700,1.878007,0.402300,0.172600,0.334100,0.334200,16.337500
7,2.029700,1.873830,0.402500,0.172300,0.334700,0.334600,16.427500
8,1.989400,1.870096,0.406400,0.175700,0.336900,0.336900,16.495000
9,1.989400,1.870586,0.406100,0.176700,0.337500,0.337500,16.482500
10,1.973500,1.870062,0.405500,0.176200,0.337200,0.337000,16.473800


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=2000, training_loss=2.023044403076172, metrics={'train_runtime': 819.2354, 'train_samples_per_second': 39.061, 'train_steps_per_second': 2.441, 'total_flos': 3393518035795968.0, 'train_loss': 2.023044403076172, 'epoch': 10.0})

In [25]:
# Evaluate the model after training on test data
print("\nEvaluating model after training on test data:")
results_with_training = evaluate_model_batch(model, test_dataset, tokenizer, data_collator)
print(results_with_training)


Evaluating model after training on test data:


Evaluating: 100%|██████████| 32/32 [00:25<00:00,  1.26it/s]


{'rouge1': 0.416, 'rouge2': 0.1728, 'rougeL': 0.3384, 'rougeLsum': 0.3383, 'gen_len': 15.924}


In [26]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

events.out.tfevents.1733457648.23a5a004e2bf.494.2:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jiraheya/samsum_model_t5_small_10_epochs/commit/d9e8e03be8cfd5222d1cadf457fcb0f6b58c204b', commit_message='Training complete', commit_description='', oid='d9e8e03be8cfd5222d1cadf457fcb0f6b58c204b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jiraheya/samsum_model_t5_small_10_epochs', endpoint='https://huggingface.co', repo_type='model', repo_id='Jiraheya/samsum_model_t5_small_10_epochs'), pr_revision=None, pr_num=None)

### Testing model to check if it is pushed propoerly to huggingface hub

In [27]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model1 = AutoModelForSeq2SeqLM.from_pretrained("Jiraheya/samsum_model_t5_small_10_epochs")
tokenizer1 = AutoTokenizer.from_pretrained("Jiraheya/samsum_model_t5_small_10_epochs")

ARTICLE_TO_SUMMARIZE = "summarize: " + test_dataset[0]['dialogue']
inputs = tokenizer1([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model1.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=60)
tokenizer1.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"Amanda has Betty's number. Amanda asks Larry to call her last time we were at the park together."

In [29]:
test_dataset[0]['dialogue']

"Hannah: Hey, do you have Betty's number?\nAmanda: Lemme check\nHannah: <file_gif>\nAmanda: Sorry, can't find it.\nAmanda: Ask Larry\nAmanda: He called her last time we were at the park together\nHannah: I don't know him well\nHannah: <file_gif>\nAmanda: Don't be shy, he's very nice\nHannah: If you say so..\nHannah: I'd rather you texted him\nAmanda: Just text him 🙂\nHannah: Urgh.. Alright\nHannah: Bye\nAmanda: Bye bye"

## Model 2 - facebook/bart-large-cnn




In [15]:
# Set up model and tokenizer
checkpoint = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

### Perform Tokenization and Load Model

In [16]:
# Tokenize dataset
# Tokenize dataset
tokenized_samsum = samsum.map(preprocess_function, batched=True, remove_columns=samsum["train"].column_names)
# Set up model and move to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BartForConditionalGeneration.from_pretrained(checkpoint).to(device)

# Load test dataset
test_dataset = load_dataset("knkarthick/samsum", split="test[:250]")

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

### Create Data Collator

In [17]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

### Setup Trainer and Training Arg

In [19]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="bart-large-cnn_samsum_model_10epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    greater_is_better=True,
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_samsum["train"],
    eval_dataset=tokenized_samsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-19-b6cf260affca>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


### Evaluate Model without training on test data.

In [20]:
tokenized_test_data = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [21]:
trainer.evaluate(tokenized_test_data)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 2.1658172607421875,
 'eval_model_preparation_time': 0.0066,
 'eval_rouge1': 0.3149,
 'eval_rouge2': 0.1074,
 'eval_rougeL': 0.2309,
 'eval_rougeLsum': 0.2318,
 'eval_gen_len': 65.8067,
 'eval_runtime': 102.615,
 'eval_samples_per_second': 2.924,
 'eval_steps_per_second': 0.37}

### Model 2 Training

In [22]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,1.428222,0.006600,0.410900,0.200800,0.308400,0.308500,59.755000
2,No log,1.507952,0.006600,0.421400,0.202700,0.317500,0.317500,59.386200
3,1.217100,1.534810,0.006600,0.409300,0.194900,0.307100,0.307000,60.206200
4,1.217100,1.711368,0.006600,0.409200,0.192800,0.306700,0.306600,60.380000
5,0.651800,1.875688,0.006600,0.414900,0.193500,0.311800,0.311700,59.500000
6,0.651800,2.052059,0.006600,0.412600,0.190200,0.310700,0.310800,60.335000
7,0.651800,2.155113,0.006600,0.413800,0.191700,0.311700,0.311500,60.188800
8,0.337100,2.405093,0.006600,0.413200,0.191300,0.311600,0.311600,60.280000
9,0.337100,2.484966,0.006600,0.414600,0.189700,0.312900,0.313100,60.737500
10,0.207200,2.526015,0.006600,0.416500,0.191100,0.314200,0.314300,60.615000


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Streaming output truncated to the last 5000 lines.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Tra

TrainOutput(global_step=2000, training_loss=0.6033039894104004, metrics={'train_runtime': 3512.83, 'train_samples_per_second': 9.109, 'train_steps_per_second': 0.569, 'total_flos': 2.292309070499021e+16, 'train_loss': 0.6033039894104004, 'epoch': 10.0})

### Push To Hub

In [23]:
trainer.push_to_hub(commit_message="Training complete", tags="summarization")

events.out.tfevents.1733462373.a0be5a43542f.13014.0:   0%|          | 0.00/13.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Jiraheya/pegasus_xsum_samsum_model_10epoch/commit/241d6071169b35140ddc478f9c9c3517883ea507', commit_message='Training complete', commit_description='', oid='241d6071169b35140ddc478f9c9c3517883ea507', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Jiraheya/pegasus_xsum_samsum_model_10epoch', endpoint='https://huggingface.co', repo_type='model', repo_id='Jiraheya/pegasus_xsum_samsum_model_10epoch'), pr_revision=None, pr_num=None)

### Evaluate After Training

In [24]:
trainer.evaluate(tokenized_test_data)

Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

{'eval_loss': 1.5278205871582031,
 'eval_model_preparation_time': 0.0066,
 'eval_rouge1': 0.4158,
 'eval_rouge2': 0.2023,
 'eval_rougeL': 0.3155,
 'eval_rougeLsum': 0.3161,
 'eval_gen_len': 59.57,
 'eval_runtime': 83.5255,
 'eval_samples_per_second': 3.592,
 'eval_steps_per_second': 0.455,
 'epoch': 10.0}

### Test if model is pushed successfully to huggingface_hub

In [25]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("Jiraheya/pegasus_xsum_samsum_model_10epoch")
tokenizer = AutoTokenizer.from_pretrained("Jiraheya/pegasus_xsum_samsum_model_10epoch")

ARTICLE_TO_SUMMARIZE = "summarize: " + test_dataset[0]['dialogue']

inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=10, max_length=60)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.56M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


"Amanda doesn't have Betty's number. She wants to ask Larry to text him."